In [129]:
import Bio
import numpy as np
from Bio import SeqIO
import sys
import csv
import os
import random as random
from functions_collection_copy import *

score_matrix={'a': {'a': 0, 'c': 5, 'g': 2, 't': 5}, 'c': {'a': 5, 'c': 0, 'g': 5, 't': 2}, 'g': {'a': 2, 'c': 5, 'g': 0, 't': 5}, 't': {'a': 5, 'c': 2, 'g': 5, 't': 0}}
gap_cost=5

In [130]:
def make_related_strings(nr_of_str:int,len_of_str, degree_of_variation:float, start_string):
    string_fam_collection=[]
    first_string=[]
    if start_string==False:
        for k in range(0,int(len_of_str)):
                first_string.append(random.choice(['a','c','t','g']))
        first_string=''.join(first_string)
        string_fam_collection.append(first_string)
    else:
         first_string=start_string
         string_fam_collection.append(first_string)

    for i in range(1,nr_of_str):
        a_sequence=[]
        for element in first_string:
            if random.random() < degree_of_variation: #checking if we should change the charachter
                a_sequence.append(random.choice(['a','c','t','g']))
            else:
                 a_sequence.append(element)
        a_sequence=''.join(a_sequence)
        string_fam_collection.append(a_sequence)
    return string_fam_collection

#make_related_strings(3,12,0.1,False)


def make_strings_in_families(nr_of_fams, nr_str_pr_fam, len_of_str, internal_var_in_fams, degree_of_var_from_first_fam):
    all_strings_coll = []
    first_family = make_related_strings(nr_str_pr_fam, len_of_str, internal_var_in_fams, start_string=False)
    all_strings_coll.append(first_family)
    
    for i in range(1, int(nr_of_fams)):
        carry_over_string = first_family[0]
        mutated_carry_over = []
        for element in carry_over_string:
            if random.random() < degree_of_var_from_first_fam:  # checking if we should change the character
                mutated_carry_over.append(random.choice(['a', 'c', 't', 'g']))
            else:
                mutated_carry_over.append(element)
        mutated_carry_over = ''.join(mutated_carry_over)  # Join the list of characters into a string
        family = make_related_strings(nr_str_pr_fam, len_of_str, internal_var_in_fams,mutated_carry_over)
        all_strings_coll.append(family)
    all_strings_coll = [item for sublist in all_strings_coll for item in sublist]
    #all_strings_coll = [''.join(sublist) for sublist in all_strings_coll]
    return all_strings_coll


In [131]:
def test_many_parameters_no_internal_reps(nr_of_str=10,len_str=20,nr_of_fams=2, str_pr_fam=5, internal_var=0.1, var_between_fams=0.9,mode='random' ):
    costs_Krus=float('inf')
    costs_Prim=float('inf')
    costs_Gus=float('inf')
    if mode=='random':
        random_sequences_collection=[]
        random_pile=[]
        for j in range (0,nr_of_str):
            a_sequence=[]
            for k in range(0,len_str):
                a_sequence.append(random.choice(['a','c','t','g']))
            a_sequence=''.join(a_sequence)
            random_pile.append(a_sequence)
        random_sequences_collection.append(random_pile)
        for random_pile in random_sequences_collection:
            #print('rand: I intend to align: '+str(random_pile))
            cost=new_assembly_gradual_x(random_pile,score_matrix,gap_cost)[2]
            costs_Krus=cost
        for random_pile in random_sequences_collection:
            cost=new_assembly_Prim_x(random_pile,score_matrix,gap_cost)[2]
            costs_Prim=cost
        for random_pile in random_sequences_collection:
            #print('rand:I intend to align: '+str(random_pile))
            cost=new_assembly_Gus_x(random_pile,score_matrix,gap_cost)[2]
            costs_Gus=cost
    elif mode=='family':
        #internal_var=[internal_var]*nr_of_fams
        #var_between_fams=[var_between_fams]*nr_of_fams
        strings_in_families=make_strings_in_families(nr_of_fams,str_pr_fam,len_str,internal_var,var_between_fams)
        #print('fam:I intend to align: '+str(strings_in_families))
        cost=new_assembly_gradual_x(strings_in_families,score_matrix,gap_cost)[2]

        costs_Krus=cost

        cost=new_assembly_Prim_x(strings_in_families,score_matrix,gap_cost)[2]
        costs_Prim=cost
        
        cost=new_assembly_Gus_x(strings_in_families,score_matrix,gap_cost)[2]
        costs_Gus=cost
        
    else:
        print("that's not a valid mode, try 'family' or 'random'!")
    #result=costs_new_gradual, costs_new_one_by_one, costs_old
    #result=costs_Krus,costs_Prim,costs_Gus
    if mode=='random':
        #params="nr str: "+ str(nr_of_str)+", str len: "+str(len_str)+",mode:"+str(mode)
        #params=nr_of_str,len_str,mode
        return(nr_of_str,len_str,mode,costs_Krus,costs_Prim,costs_Gus)
    if mode=='family':
        #params="str len: "+str(len_str)+", nr fams: "+str(nr_of_fams)+", str_pr_fam: "+str(str_pr_fam)+", internal var: "+ str(internal_var)+", var between fams: "+str(var_between_fams)+", mode:"+str(mode)
        return (len_str,nr_of_fams,str_pr_fam,internal_var,var_between_fams,mode,costs_Krus,costs_Prim,costs_Gus)

test_many_parameters_no_internal_reps(len_str=15,nr_of_fams=3,str_pr_fam=3,internal_var=0.1,var_between_fams=0.9, mode='random')


    
        

(10, 15, 'random', 2802, 2698, 2219)

In [132]:
#for random costs
repeats=3
mode='random'
str_len=[10,50,100,200,300,500,750,1000]
nr_str=[5,10,15]

csv_file_path="testing_outputs/random_strs_tests.csv"
with open(csv_file_path, 'a', newline='') as csvfile:
                            csvwriter = csv.writer(csvfile, delimiter=',', quotechar=' ', quoting=csv.QUOTE_NONE)
                            csvwriter.writerow(["nr_of_str","str_len","mode","Kruskal","Prim","Gusfield"])

for i in range(0,repeats):
    for lens in str_len:
        for nr in nr_str:

            nr_of_str,len_str,mode,costs_Krus,costs_Prim,costs_Gus=test_many_parameters_no_internal_reps(nr,lens,mode='random')
            with open(csv_file_path, 'a', newline='') as csvfile:
                csvwriter = csv.writer(csvfile)
                csvwriter.writerow([nr_of_str,len_str,mode,costs_Krus,costs_Prim,costs_Gus])

In [150]:
#for families cost
repeats=3
mode='family'
nr_fams=[1,2,4]
str_pr_fam=[3,10]
internal_var=[0.1,0.9]
var_between_fams=[0.1,0.9]
str_len=[10,50,100,200,400]
for x in var_between_fams:
    print(x)
csv_file_path="testing_outputs/familiy_outputs_testing2.csv"
with open(csv_file_path, 'a', newline='') as csvfile:
                            csvwriter = csv.writer(csvfile)
                            csvwriter.writerow(["str_len","nr_of_fams","str_pr_fam","internal_var","external_var","mode","Kruskal","Prim","Gusfield"])

for i in range(0,repeats):
    for fam_num in nr_fams:
        for str_pr_fam_nr in str_pr_fam:
            for int_var in internal_var:
                for ex_var in var_between_fams:
                    for str_l in str_len:

                        len_str,nr_of_fams,strs_pr_fam,inte_var,var_between_fam,mode,costs_Krus,costs_Prim,costs_Gus=test_many_parameters_no_internal_reps(len_str=str_l,nr_of_fams=fam_num,internal_var=int_var,str_pr_fam=str_pr_fam_nr,var_between_fams=ex_var,mode=mode)

                        with open(csv_file_path, 'a', newline='') as csvfile:
                            csvwriter = csv.writer(csvfile)
                            csvwriter.writerow([len_str,nr_of_fams,strs_pr_fam,inte_var,var_between_fam,mode,costs_Krus,costs_Prim,costs_Gus])

0.1
0.9


































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































KeyboardInterrupt: 

In [142]:
import time
import random


def test_many_parameters_no_internal_reps_time(nr_of_str=10, len_str=20, nr_of_fams=2, str_pr_fam=5, internal_var=0.1, var_between_fams=0.9, mode='random'):
    time_taken_Krus = float('inf')
    time_taken_Prim = float('inf')
    time_taken_Gus = float('inf')

    start_time = time.time()

    if mode == 'random':
        random_sequences_collection = []
        random_pile = []
        for j in range(0, nr_of_str):
            a_sequence = []
            for k in range(0, len_str):
                a_sequence.append(random.choice(['a', 'c', 't', 'g']))
            a_sequence = ''.join(a_sequence)
            random_pile.append(a_sequence)
        random_sequences_collection.append(random_pile)
        for random_pile in random_sequences_collection:
            start_func_time = time.time()
            new_assembly_gradual_x(random_pile, score_matrix, gap_cost)
            end_func_time = time.time()
            time_taken_Krus = end_func_time - start_func_time

        for random_pile in random_sequences_collection:
            start_func_time = time.time()
            new_assembly_Prim_x(random_pile, score_matrix, gap_cost)
            end_func_time = time.time()
            time_taken_Prim = end_func_time - start_func_time

        for random_pile in random_sequences_collection:
            start_func_time = time.time()
            new_assembly_Gus_x(random_pile, score_matrix, gap_cost)
            end_func_time = time.time()
            time_taken_Gus = end_func_time - start_func_time

    elif mode == 'family':
        strings_in_families = make_strings_in_families(nr_of_fams, str_pr_fam, len_str, internal_var, var_between_fams)
        start_func_time = time.time()
        new_assembly_gradual_x(strings_in_families, score_matrix, gap_cost)
        end_func_time = time.time()
        time_taken_Krus = end_func_time - start_func_time

        start_func_time = time.time()
        new_assembly_Prim_x(strings_in_families, score_matrix, gap_cost)
        end_func_time = time.time()
        time_taken_Prim = end_func_time - start_func_time

        start_func_time = time.time()
        new_assembly_Gus_x(strings_in_families, score_matrix, gap_cost)
        end_func_time = time.time()
        time_taken_Gus = end_func_time - start_func_time

    else:
        print("That's not a valid mode, try 'family' or 'random'!")

    
    if mode == 'random':
        return nr_of_str, len_str, mode, time_taken_Krus, time_taken_Prim, time_taken_Gus
    if mode == 'family':
        return len_str, nr_of_fams, str_pr_fam, internal_var, var_between_fams, mode, time_taken_Krus, time_taken_Prim, time_taken_Gus

# Example usage
(test_many_parameters_no_internal_reps_time(len_str=15, nr_of_fams=3, str_pr_fam=3, internal_var=0.1, var_between_fams=0.9, mode='family'))


(15,
 3,
 3,
 0.1,
 0.9,
 'family',
 0.23749303817749023,
 0.2776970863342285,
 0.19854950904846191)

In [153]:
#for random time
repeats=3
mode='random'
str_len=[10,50,100,200,300,500,1000]
nr_str=[5,10,15]

csv_file_path="testing_outputs/random_strs_tests_time.csv"
with open(csv_file_path, 'a', newline='') as csvfile:
                            csvwriter = csv.writer(csvfile, delimiter=',', quotechar=' ', quoting=csv.QUOTE_NONE)
                            csvwriter.writerow(["nr_of_str","str_len","mode","Kruskal","Prim","Gusfield"])

for i in range(0,repeats):
    for lens in str_len:
        for nr in nr_str:

            nr_of_str,len_str,mode,time_Krus,time_Prim,time_Gus=test_many_parameters_no_internal_reps_time(nr,lens,mode='random')
            with open(csv_file_path, 'a', newline='') as csvfile:
                csvwriter = csv.writer(csvfile)
                csvwriter.writerow([nr_of_str,len_str,mode,time_Krus,time_Prim,time_Gus])

In [154]:
#for familiy time

repeats=3
mode='family'
nr_fams=[1,2,4]
str_pr_fam=[3,10]
internal_var=[0.1,0.9]
var_between_fams=[0.1,0.9]
str_len=[10,50,100,200]
for x in var_between_fams:
    print(x)
csv_file_path="testing_outputs/familiy_outputs_testing_time.csv"
with open(csv_file_path, 'a', newline='') as csvfile:
                            csvwriter = csv.writer(csvfile)
                            csvwriter.writerow(["str_len","nr_of_fams","str_pr_fam","internal_var","external_var","mode","Kruskal","Prim","Gusfield"])

for i in range(0,repeats):
    for fam_num in nr_fams:
        for str_pr_fam_nr in str_pr_fam:
            for int_var in internal_var:
                for ex_var in var_between_fams:
                    for str_l in str_len:

                        len_str,nr_of_fams,strs_pr_fam,inte_var,var_between_fam,mode,time_Krus,time_Prim,time_Gus=test_many_parameters_no_internal_reps_time(len_str=str_l,nr_of_fams=fam_num,str_pr_fam=str_pr_fam_nr,internal_var=int_var,var_between_fams=ex_var,mode=mode)
                        with open(csv_file_path, 'a', newline='') as csvfile:
                            csvwriter = csv.writer(csvfile)
                            csvwriter.writerow([len_str,nr_of_fams,strs_pr_fam,inte_var,var_between_fam,mode,time_Krus,time_Prim,time_Gus])

0.1
0.9
































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































